![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Taller: Construcción e implementación de modelos Bagging, Random Forest y XGBoost

En este taller podrán poner en práctica sus conocimientos sobre la construcción e implementación de modelos de Bagging, Random Forest y XGBoost. El taller está constituido por 8 puntos, en los cuales deberan seguir las intrucciones de cada numeral para su desarrollo.

## Datos predicción precio de automóviles

En este taller se usará el conjunto de datos de Car Listings de Kaggle donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

In [17]:
#pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
# Importación de librerías
%matplotlib inline
import pandas as pd

# Lectura de la información de archivo .csv
data = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')

# Preprocesamiento de datos para el taller
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data = data.drop(['Model'], axis=1)

# Visualización dataset
data.head()

,Price,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE
7,21995,2014,6480,0,0,0,1,0,0,0
11,13995,2014,39972,0,0,0,0,1,0,0
167,17941,2016,18989,0,0,0,0,0,1,0
225,12493,2014,51330,0,0,0,1,0,0,0
270,7994,2007,116065,0,1,0,0,0,0,0


In [23]:
# Separación de variables predictoras (X) y variable de interés (y)
y = data['Price']
X = data.drop(['Price'], axis=1)

In [24]:
# Separación de datos en set de entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Punto 1 - Árbol de decisión manual

En la celda 1 creen un árbol de decisión **manualmente**  que considere los set de entrenamiento y test definidos anteriormente y presenten el RMSE y MAE del modelo en el set de test.

In [25]:
# Celda 1
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Definición del árbol de decisión manual
def decision_tree_manual(X_train, y_train, X_test):
    # Reglas de división
    threshold_year = 2015
    threshold_mileage = 50000
    
    # Predicción para el conjunto de prueba
    y_pred = []
    for index, row in X_test.iterrows():
        if row['Year'] <= threshold_year:
            if row['Mileage'] <= threshold_mileage:
                y_pred.append(np.mean(y_train[(X_train['Year'] <= threshold_year) & (X_train['Mileage'] <= threshold_mileage)]))
            else:
                y_pred.append(np.mean(y_train[(X_train['Year'] <= threshold_year) & (X_train['Mileage'] > threshold_mileage)]))
        else:
            if row['Mileage'] <= threshold_mileage:
                y_pred.append(np.mean(y_train[(X_train['Year'] > threshold_year) & (X_train['Mileage'] <= threshold_mileage)]))
            else:
                y_pred.append(np.mean(y_train[(X_train['Year'] > threshold_year) & (X_train['Mileage'] > threshold_mileage)]))
    
    return np.array(y_pred)

# Predicción utilizando el árbol de decisión manual
y_pred_manual = decision_tree_manual(X_train, y_train, X_test)

# Cálculo de RMSE y MAE
rmse_manual = np.sqrt(mean_squared_error(y_test, y_pred_manual))
mae_manual = mean_absolute_error(y_test, y_pred_manual)

print("RMSE del árbol de decisión manual:", rmse_manual)
print("MAE del árbol de decisión manual:", mae_manual)


RMSE del árbol de decisión manual: 2584.577836553867
MAE del árbol de decisión manual: 1998.9576329972087


### Punto 2 - Bagging manual

En la celda 2 creen un modelo bagging **manualmente** con 10 árboles de regresión y comenten sobre el desempeño del modelo.

In [26]:
# Celda 2
from sklearn.tree import DecisionTreeRegressor

# Definición del modelo Bagging manual
def bagging_manual(X_train, y_train, X_test, num_trees=10):
    y_preds = []
    
    for _ in range(num_trees):
        # Crear un subconjunto aleatorio del conjunto de entrenamiento (muestreo con reemplazo)
        idx = np.random.choice(X_train.index, size=len(X_train), replace=True)
        X_train_subset = X_train.loc[idx]
        y_train_subset = y_train.loc[idx]
        
        # Entrenar un árbol de decisión en el subconjunto
        tree = DecisionTreeRegressor(random_state=42)
        tree.fit(X_train_subset, y_train_subset)
        
        # Realizar predicciones en el conjunto de prueba
        y_pred = tree.predict(X_test)
        y_preds.append(y_pred)
    
    # Promediar las predicciones de todos los árboles
    y_pred_bagging = np.mean(y_preds, axis=0)
    
    return y_pred_bagging

# Predicción utilizando el modelo Bagging manual
y_pred_bagging_manual = bagging_manual(X_train, y_train, X_test, num_trees=10)

# Cálculo de RMSE y MAE
rmse_bagging_manual = np.sqrt(mean_squared_error(y_test, y_pred_bagging_manual))
mae_bagging_manual = mean_absolute_error(y_test, y_pred_bagging_manual)

#print("RMSE del modelo Bagging manual:", rmse_bagging_manual)
#print("MAE del modelo Bagging manual:", mae_bagging_manual)
print("RMSE del modelo Bagging manual: {:.3f}".format(rmse_bagging_manual))
print("MAE del modelo Bagging manual: {:.3f}".format(mae_bagging_manual))

RMSE del modelo Bagging manual: 1796.967
MAE del modelo Bagging manual: 1338.846


### Punto 3 - Bagging con librería

En la celda 3, con la librería sklearn, entrenen un modelo bagging con 10 árboles de regresión y el parámetro `max_features` igual a `log(n_features)` y comenten sobre el desempeño del modelo.

In [27]:
# Celda 3
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from math import log2

# Definición del modelo Bagging con la librería sklearn
bagging_model_sklearn = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_features=int(log2(X_train.shape[1]))), 
                                         n_estimators=10, 
                                         random_state=42)

# Entrenamiento del modelo
bagging_model_sklearn.fit(X_train, y_train)

# Predicción en el conjunto de prueba
y_pred_bagging_sklearn = bagging_model_sklearn.predict(X_test)

# Cálculo de RMSE y MAE
rmse_bagging_sklearn = np.sqrt(mean_squared_error(y_test, y_pred_bagging_sklearn))
mae_bagging_sklearn = mean_absolute_error(y_test, y_pred_bagging_sklearn)

print("RMSE del modelo Bagging con sklearn:", rmse_bagging_sklearn)
print("MAE del modelo Bagging con sklearn:", mae_bagging_sklearn)


RMSE del modelo Bagging con sklearn: 1819.7321845417173
MAE del modelo Bagging con sklearn: 1361.2664479544706


### Punto 4 - Random forest con librería

En la celda 4, usando la librería sklearn entrenen un modelo de Randon Forest para regresión  y comenten sobre el desempeño del modelo.

In [28]:
# Celda 4
from sklearn.ensemble import RandomForestRegressor

# Definición del modelo Random Forest con la librería sklearn
random_forest_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Entrenamiento del modelo
random_forest_model.fit(X_train, y_train)

# Predicción en el conjunto de prueba
y_pred_random_forest = random_forest_model.predict(X_test)

# Cálculo de RMSE y MAE
rmse_random_forest = np.sqrt(mean_squared_error(y_test, y_pred_random_forest))
mae_random_forest = mean_absolute_error(y_test, y_pred_random_forest)

print("RMSE del modelo Random Forest:", rmse_random_forest)
print("MAE del modelo Random Forest:", mae_random_forest)


RMSE del modelo Random Forest: 1765.4118259983413
MAE del modelo Random Forest: 1314.4207078056425


### Punto 5 - Calibración de parámetros Random forest

En la celda 5, calibren los parámetros max_depth, max_features y n_estimators del modelo de Randon Forest para regresión, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [29]:
# Celda 5
from sklearn.model_selection import GridSearchCV

# Definición del modelo Random Forest
random_forest_model = RandomForestRegressor(random_state=42)

# Definición de los parámetros a ajustar
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2'],
    'n_estimators': [50, 100, 200]
}

# Búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(estimator=random_forest_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mejor combinación de parámetros
best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)

# Mejor modelo después de la búsqueda de hiperparámetros
best_random_forest_model = grid_search.best_estimator_

# Predicción en el conjunto de prueba con el mejor modelo
y_pred_best_random_forest = best_random_forest_model.predict(X_test)

# Cálculo de RMSE y MAE con el mejor modelo
rmse_best_random_forest = np.sqrt(mean_squared_error(y_test, y_pred_best_random_forest))
mae_best_random_forest = mean_absolute_error(y_test, y_pred_best_random_forest)

print("RMSE del mejor modelo Random Forest:", rmse_best_random_forest)
print("MAE del mejor modelo Random Forest:", mae_best_random_forest)


Mejores parámetros: {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 200}
RMSE del mejor modelo Random Forest: 1564.2461359342767
MAE del mejor modelo Random Forest: 1147.2014922680428


### Punto 6 - XGBoost con librería

En la celda 6 implementen un modelo XGBoost de regresión con la librería sklearn y comenten sobre el desempeño del modelo.

In [30]:
# Celda 6
from xgboost import XGBRegressor

# Definición del modelo XGBoost con la librería sklearn
xgb_model = XGBRegressor(random_state=42)

# Entrenamiento del modelo
xgb_model.fit(X_train, y_train)

# Predicción en el conjunto de prueba
y_pred_xgb = xgb_model.predict(X_test)

# Cálculo de RMSE y MAE
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)

print("RMSE del modelo XGBoost:", rmse_xgb)
print("MAE del modelo XGBoost:", mae_xgb)


RMSE del modelo XGBoost: 1605.2355199928948
MAE del modelo XGBoost: 1185.2272991506386


### Punto 7 - Calibración de parámetros XGBoost

En la celda 7 calibren los parámetros learning rate, gamma y colsample_bytree del modelo XGBoost para regresión, comenten sobre el desempeño del modelo y describan cómo cada parámetro afecta el desempeño del modelo.

In [31]:
# Celda 7
from sklearn.model_selection import GridSearchCV

# Definición del modelo XGBoost
xgb_model = XGBRegressor(random_state=42)

# Definición de los parámetros a ajustar
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'gamma': [0, 0.1, 0.2],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mejor combinación de parámetros
best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)

# Mejor modelo después de la búsqueda de hiperparámetros
best_xgb_model = grid_search.best_estimator_

# Predicción en el conjunto de prueba con el mejor modelo
y_pred_best_xgb = best_xgb_model.predict(X_test)

# Cálculo de RMSE y MAE con el mejor modelo
rmse_best_xgb = np.sqrt(mean_squared_error(y_test, y_pred_best_xgb))
mae_best_xgb = mean_absolute_error(y_test, y_pred_best_xgb)

print("RMSE del mejor modelo XGBoost:", rmse_best_xgb)
print("MAE del mejor modelo XGBoost:", mae_best_xgb)


Mejores parámetros: {'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.1}
RMSE del mejor modelo XGBoost: 1543.9585735005624
MAE del mejor modelo XGBoost: 1134.757230763072


### Punto 8 - Comparación y análisis de resultados
En la celda 8 comparen los resultados obtenidos de los diferentes modelos (random forest y XGBoost) y comenten las ventajas del mejor modelo y las desventajas del modelo con el menor desempeño.

# Celda 8
Con los resultados arrojados podemos realizar la comparación y el análisis de los diferentes modelos:

Análisis y comparación:
- Mejor rendimiento (menor RMSE): El modelo XGBoost con calibración de parámetros (RMSE: 1543.96).
- Mejor rendimiento (menor MAE): El modelo XGBoost con calibración de parámetros (MAE: 1134.76).

Ventajas del mejor modelo (XGBoost con calibración de parámetros):
- Proporciona un RMSE y MAE más bajos en comparación con otros modelos, lo que indica una mejor capacidad de predicción.
- XGBoost tiene una capacidad de regularización incorporada y puede manejar relaciones más complejas entre las características.
- La calibración de parámetros ayuda a ajustar el modelo para obtener el mejor rendimiento posible.
- Desventajas del modelo con menor desempeño (Árbol de decisión manual):
- El rendimiento del árbol de decisión manual es el peor en términos de RMSE y MAE, lo que indica que es el modelo menos preciso.
- Los árboles de decisión manuales pueden ser propensos al sobreajuste y tienen una capacidad limitada para capturar relaciones complejas en los datos.
- En resumen, el modelo XGBoost con calibración de parámetros muestra el mejor rendimiento en términos de precisión predictiva en este conjunto de datos específico. Proporciona resultados significativamente mejores que los otros modelos considerados.

In [35]:
import pandas as pd

# Datos de los modelos
datos_modelos = {
    "Árbol de decisión manual": [2584.58, 1998.96],
    "Bagging manual": [1792.60, 1337.23],
    "Bagging con librería (Sklearn)": [1819.73, 1361.27],
    "Random Forest con librería (Sklearn)": [1765.41, 1314.42],
    "Random Forest con calibración de parámetros": [1564.25, 1147.20],
    "XGBoost con librería (Sklearn)": [1605.24, 1185.23],
    "XGBoost con calibración de parámetros": [1543.96, 1134.76]
}

# Convertir el diccionario en un DataFrame de Pandas
df_modelos = pd.DataFrame.from_dict(datos_modelos, orient='index', columns=['RMSE', 'MAE'])

# Agregar el encabezado "Nombre modelo" a la primera columna
df_modelos.index.name = 'Ressultado nombre modelo'
#print(df_modelos)
print(df_modelos.to_markdown())


| Ressultado nombre modelo                    |    RMSE |     MAE |
|:--------------------------------------------|--------:|--------:|
| Árbol de decisión manual                    | 2584.58 | 1998.96 |
| Bagging manual                              | 1792.6  | 1337.23 |
| Bagging con librería (Sklearn)              | 1819.73 | 1361.27 |
| Random Forest con librería (Sklearn)        | 1765.41 | 1314.42 |
| Random Forest con calibración de parámetros | 1564.25 | 1147.2  |
| XGBoost con librería (Sklearn)              | 1605.24 | 1185.23 |
| XGBoost con calibración de parámetros       | 1543.96 | 1134.76 |
